## CPSC 340 Lecture 21: kernel methods

This notebook is for the in-class activities. It assumes you have already watched the [associated video](https://www.youtube.com/watch?v=mba4xweShwI&list=PLWmXHcz_53Q02ZLeAxigki1JZFfCO6M-b&index=21).

<font color='red'>**REMINDER TO START RECORDING**</font>

Also, reminder to enable screen sharing for Participants.

## Pre-class music



1. Sweet Baby! by Mere Notilde
2. THE BADDEST by K/DA
3. 911 by Lady Gaga

## Admin

- a4 due tonight
- I updated the schedule, Friday's lecture will be on stochastic gradient descent
- Some of the toughest parts of the class are happening now (softmax, kernels, MLE/MAP, PCA)

## Video chapters

- linear SVM minutiae
- n-by-n normal equations
- feature transformation demo
- multi-dimensional polynomial basis
- kernel trick
- linear regression vs. kernel regression
- Gaussian RBF kernel
- kernel trick applications
- RBF SVM hyperparameters demo
- RBF SVM minutiae
- summary

![](img/L21/image.png)

## Old exam questions

#### 2016W2 final

Name an advantage and a disadvantage of using an SVM with the RBF kernel instead of a linear kernel.

**Mike's answer:**

- Advantage of RBF (vs. linear): fit complex, nonlinear decision boundaries
- Disadvantage of RBF (vs. linear): slow, non-parametric - need to store (part of) dataset, prone to overfitting

#### 2018W1 final

![](img/L21/matching2_top.png)

![](img/L21/matching2_bottom.png)

**Mike's answers:**

- i) KNN regression 
- ii) OLS
- iii) polynomial, because it doesn't go to zero away from the data
- iv) RBF with smaller width
- v) ridge regression (OLS + L2)
- vi) RBF with larger width

## Demo of the "other" normal equations and the kernel trick

Consider the randomly generated dataset below:

In [48]:
import numpy as np
n = 20
d = 3
p = 2 
λ = 1
np.random.seed(1)
X = np.random.randint(0,10,size=(n,d))
X

array([[5, 8, 9],
       [5, 0, 0],
       [1, 7, 6],
       [9, 2, 4],
       [5, 2, 4],
       [2, 4, 7],
       [7, 9, 1],
       [7, 0, 6],
       [9, 9, 7],
       [6, 9, 1],
       [0, 1, 8],
       [8, 3, 9],
       [8, 7, 3],
       [6, 5, 1],
       [9, 3, 4],
       [8, 1, 4],
       [0, 3, 9],
       [2, 0, 4],
       [9, 2, 7],
       [7, 9, 8]])

In [49]:
y = np.random.randint(-5, 5, n)
y

array([ 1,  4, -2,  2,  2, -1,  0,  4, -2,  1,  3, -5, -3,  2,  2,  4,  2,
       -2, -5,  3])

We decide to use a degree 2 polynomial basis to fit this dataset, using L2 regularization with $\lambda=1$. 

We then want to make a prediction on the test vector:

In [50]:
Xtest = np.array([[-1, 0, 1]])

Code this up 3 ways:

1. Using the "regular" normal equations
2. Using the "other" normal equations without the kernel trick
3. Using the "other" normal equations with the kernel trick

and show that you get the same answers all 3 ways.

In [51]:
from sklearn.preprocessing import PolynomialFeatures

#### Approach 1

In [52]:
poly = PolynomialFeatures(degree=p)
Z = poly.fit_transform(X)

In [53]:
Z.shape

(20, 10)

Running time of the above step: 

- We have $O(d^p)$ features we're making. Let's call this $O(k)$ for now to keep it general and we can plug $k=d^p$ back in at the end.
- We need to transform each of the $n$ training examples. 
- For each training example, for each new feature, we potentially need to look at all the old features.
- $O(ndk)$

In [54]:
w = np.linalg.solve(Z.T@Z + λ*np.eye(poly.n_output_features_), Z.T@y)
w

array([-1.325003  ,  1.62053602, -1.77771719,  0.1154218 , -0.02284804,
       -0.0384322 , -0.22760804,  0.13310881,  0.11101713,  0.04284918])

Running time of the above step:

- Multiply $Z^TZ$, meaning an $k \times n$ multiplied by $n \times k$ which is $O(nk^2)$
- Then the solving is $O(k^3)$ so we have $O(nk^2 + k^3)$ which we've seen in an earlier lecture.

In [55]:
Ztest = poly.transform(Xtest)
Ztest@w

array([-2.58250803])

Running time of the above step:

- $O(k)$
- So the total is $O(ndk+nk^2+k^3)$.

#### Approach 2

In [56]:
w_other = Z.T @ np.linalg.solve(Z@Z.T + λ*np.eye(n), y)
w_other

array([-1.325003  ,  1.62053602, -1.77771719,  0.1154218 , -0.02284804,
       -0.0384322 , -0.22760804,  0.13310881,  0.11101713,  0.04284918])

Running time of the above step:

- Multiply $ZZ^T$, meaning an $n \times k$ multiplied by $k \times n$ which is $O(n^2k)$
- Then the solving is $O(n^3)$ so we have $O(n^2k + n^3)$ which we've seen in the lecture.

We can see we get the same $w$ so the prediction will also be the same.

In [57]:
Ztest = poly.transform(Xtest)
Ztest@w_other

array([-2.58250803])

#### Approach 3

$K(a, b) = (1+a^Tb)^p$

In [58]:
K = (1 + np.sum(X[None] * X[:,None], axis=-1))**p

In [59]:
K.shape

(20, 20)

- $K$ is $n \times n$ so it costs $O(n^2d)$

In [60]:
u = np.linalg.solve(K + λ*np.eye(n), y)

In [61]:
u.shape

(20,)

- Costs $n^3$ to solve the $n \times n$ system.

In [62]:
Ktest = (1 + np.sum(X[None] * Xtest[:,None], axis=-1))**p

In [63]:
Ktest.shape

(1, 20)

In [64]:
Ktest @ u

array([-2.8150272])

And we get the same prediction!

- Total cost $O(n^2d + n^3)$ which is independent of $k$.

Summary of running times:

- Regular approach $O(ndk+nk^2+k^3)$ 
- Other normal equations $O(ndk + n^2k + n^3)$
- Kernel trick $O(n^2d + n^3)$
  - Assuming kernel function costs $O(d)$ per pair

Some sanity checks:

In [18]:
from sklearn.metrics import pairwise_kernels

In [19]:
from sklearn.kernel_ridge import KernelRidge
kr = KernelRidge(kernel="polynomial", degree=2, alpha=1, coef0=1, gamma=1)
kr.fit(X, y)

KernelRidge(degree=2, gamma=1, kernel='polynomial')

In [20]:
kr.dual_coef_

array([-0.31338522, -2.39827335, -1.45403079,  0.47762859,  1.67957059,
       -0.31214823, -0.83869815,  2.29852489, -0.65777287,  0.9480587 ,
        1.36667025, -1.13043066, -2.67360463,  1.85310435,  1.52846018,
        1.39445037, -0.03268335, -3.24799773, -2.71651028,  2.73737419])

In [21]:
kr.predict(Xtest)

array([-2.8150272])

In [22]:
# pairwise_kernels(X, X, metric="polynomial", degree=2, coef0=1, gamma=1)

## Question

What is the **space complexity** of a trained model for each of the above 3 approaches?

**Mike's answer:**

- Approach 1: $O(k)$, just storing $w$
- Approach 2: $O(k)$, same as above
- Approach 3: $O(nd)$, need to store the training data

## Question

What is the **prediction time complexity** (for one example) for each of the above 3 approaches?

**Mike's answer:**

- Approach 1: $O(kd)$, the time to form `Ztest`. There's also the dot product which takes $O(k)$ but that's negligible.
- Approach 2: $O(kd)$, same as above
- Approach 3: $O(nd)$, need to run through the training data